# NLP for Movie Plots
*Hongyuan Jin, Eunseo Kang, Paige Harris, and Viki Papadakis*

## Purpose
We use natural language processing tools to find similar movies based on their summary plots.

## Methodology
- TF-IDF
- LDA

## WIP - improvements
Todos:
- replace character names with 'character'
- verify the clustering efficiency
-

## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

## Library import for Python
We import all the required Python libraries

In [187]:
import os
# print("Current Working Directory " , os.getcwd())
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
import plotly
import plotly.graph_objs as go
import plotly.offline as ply
plotly.offline.init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

import matplotlib as plt

import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
import re
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

Current Working Directory  /Users/aj/PycharmProjects/film_python


## Library import for R
We enable R in the notebook and import all the required libraries.

In [167]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [168]:
%%R
# install.packages("ggplot2", repos='http://cran.us.r-project.org', quiet=TRUE)
library(ggplot2)
library(fixest)
library(dplyr)
library(tidyr)
library(tidyverse)
# install.packages("lmtest")
library(lmtest)
library(sandwich)

R[write to console]: trying URL 'https://cloud.r-project.org/bin/macosx/contrib/4.0/lmtest_0.9-38.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 399050 bytes (389 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write


The downloaded binary packages are in
	/var/folders/_5/gg08zw2j57j_yx44r4y359p00000gn/T//Rtmpb1Q609/downloaded_packages


## Local library import
We import all the required local libraries

In [15]:
# Include local library paths
import sys
# sys.path.append('path/to/local/lib') # uncomment and fill to import local libraries

# Import local libraries

# Data import
We first retrieve the data for movie summary plots.
*Source:*

In [16]:
movies_df = pd.read_csv('~/Dropbox/Course/PSTAT234/spr21/pstat234-final-project/wiki_movie_plots_deduped.csv')
movies_df.head(5)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Unnamed: 8
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",NaN
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",NaN
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",NaN
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,NaN
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,NaN


We subset the data for movies produced in the US and reindex the data.

In [25]:
movies_df_sub = movies_df[(movies_df['Origin/Ethnicity']=="American")].reset_index(drop=True)
# movies_df_sub = movies_df[(movies_df['Release Year']=='2000')
#                           & (movies_df['Origin/Ethnicity']=="American")].reset_index(drop=True)

# Data processing for movie plots

# NLP
We use TF-IDF and LDA for our plot summary analysis.

## TF-IDF

In [26]:
stemmer = SnowballStemmer("english", ignore_stopwords=False)

normalized = []
totalvocab_tokenized = []
def normalize(X):
    normalized = []
    # totalvocab_tokenized = []
    for x in X:
        words = nltk.word_tokenize(x)
        normalized.append(' '.join([stemmer.stem(word) for word in words if re.match('[a-zA-Z]+', word)]))
        totalvocab_tokenized.extend(words)
    return normalized

pipe = Pipeline([
    ('normalize', FunctionTransformer(normalize, validate=False)),
    ('counter_vectorizer', CountVectorizer(
        max_df=0.8, max_features=200000,
        min_df=0.2, stop_words='english',
        ngram_range=(1, 3)
    )),
    ('tfidf_transform', TfidfTransformer())
])

tfidf_matrix = pipe.fit_transform([x for x in movies_df_sub['Plot']])
print(tfidf_matrix.shape)

(17377, 78)


In [27]:
similarity_distance = 1 - cosine_similarity(tfidf_matrix)

We test for finding similar movies for a given title

In [28]:
def find_similar(title):
    index = movies_df_sub[movies_df_sub['Title'] == title].index[0]
    vector = similarity_distance[index, :]
    most_similar = movies_df_sub.iloc[np.argsort(vector)[0:10], 0:2]
    return most_similar

The movies similar to *Madagascar (2005)* according to TF-IDF.

In [29]:
print(find_similar('Madagascar'))

      Release Year                       Title
14711         2005                  Madagascar
13292         1998                  The Mighty
12531         1995  The Pebble and the Penguin
13112         1997             Robinson Crusoe
14944         2006                  Happy Feet
10331         1985                      Brazil
17171         2017  xXx: Return of Xander Cage
14384         2003                  Wrong Turn
10034         1982                        Tron
12924         1997         Beverly Hills Ninja


#### Visualize the movies within a certain similarity range
TBU

cluster the movies based on the similarity metric

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 23
# print(num_clusters)
km = KMeans(n_clusters=num_clusters)

km.fit(tfidf_matrix)

clusters_tfidf = km.labels_.tolist()

import joblib

#uncomment the below to save your model
#since I've already run my model I am loading from the pickle

joblib.dump(km, 'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

films = {'title': movies_df_sub["Title"], 'Release_year':  movies_df_sub["Release Year"],
         'Plot':  movies_df_sub['Plot'], 'cluster': clusters_tfidf,
         'Director':  movies_df_sub['Director'], 'Genre':  movies_df_sub['Genre']}
# print(films)
frame = pd.DataFrame(data=films, columns=['title', 'cluster', 'Genre', 'Release_year', 'Director'])
# print(frame)
print(frame['cluster'].value_counts())
# frame.to_csv(r'~/Dropbox/ImmInnHollywood/data/wiki_film_plot/plot_cluster.csv', index=False)

## LDA

TBU-Text explanation

In [243]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

processed_docs = movies_df_sub['Plot'].map(preprocess)
processed_docs[:10]

dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 15:
        break

dictionary.filter_extremes(no_below=20, no_above=0.1, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=23, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=23, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

mdiff, annotation = lda_model_tfidf.diff(lda_model_tfidf)
for index, score in sorted(lda_model_tfidf[bow_corpus[4]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

0 appear
1 assault
2 bartend
3 beer
4 begin
5 break
6 bucket
7 burst
8 cash
9 charact
10 custom
11 drink
12 dump
13 everybodi
14 eye
15 face
[(0, 0.13227954703913156),
 (1, 0.3790397673199794),
 (2, 0.374438855287237),
 (3, 0.2172397731360707),
 (4, 0.15173497760613988),
 (5, 0.1399498994366303),
 (6, 0.14142404420842042),
 (7, 0.14506212816523342),
 (8, 0.1793527179919929),
 (9, 0.13013718763569648),
 (10, 0.1350117256373554),
 (11, 0.3686891209949009),
 (12, 0.16259719029079714),
 (13, 0.26063277479936975),
 (14, 0.18848108896763566),
 (15, 0.20067540564150227),
 (16, 0.17125349985356006),
 (17, 0.1172126069805695),
 (18, 0.15900581156216126),
 (19, 0.19577237362575506),
 (20, 0.22411229640680644),
 (21, 0.10452823017669358),
 (22, 0.1628664933090168)]
Topic: 0 
Words: 0.014*"alic" + 0.006*"shaw" + 0.003*"doyl" + 0.003*"mitch" + 0.003*"gang" + 0.002*"kid" + 0.002*"door" + 0.002*"suzi" + 0.002*"parent" + 0.002*"ellen"
Topic: 1 
Words: 0.007*"band" + 0.006*"ethan" + 0.005*"logan" + 0.0

## Import data for names and replace the character names with string 'character'

We find that the topic modeling is affected by the character names.
Below we will replace the names with the 'Character'

*Data source:*

In [251]:
filename = '~/Dropbox/Course/PSTAT234/spr21/pstat234-final-project/yob1980.txt'
names = pd.read_csv(filename, delimiter = ",", header=None)
names.columns = ["name", 'gender', 'count']
names.head(5)
names = names.sort_values('count')
# keep the names with more than 1000 counts
# names = names[names['count']>=100]
print(names)

              name gender  count
19456          Zoe      M      5
11575      Natacia      F      5
11574      Nasreen      F      5
11573     Nashanda      F      5
11572       Narine      F      5
...            ...    ...    ...
12165        David      M  41920
12164        Jason      M  48170
12163  Christopher      M  49096
0         Jennifer      F  58379
12162      Michael      M  68704

[19457 rows x 3 columns]


In [252]:
name = names.name.tolist()
# print(name)
'Alice' in name

True

In [255]:
# load names.zip which has more names data txt files
import zipfile
with zipfile.ZipFile('/Users/aj/Dropbox/Course/PSTAT234/spr21/pstat234-final-project/names.zip', 'r') as zip_ref:
    zip_ref.extractall('names/')

In [318]:
from pathlib import Path
allnames_files = Path('names').glob('yob*')
# print(allnames_files)
# allnames_files.sort()
allnames_list = list()

for f in allnames_files:
    step_1 = pd.read_csv(f, delimiter = ',', header=None)
    allnames_list.append(step_1)
allnames = pd.concat(allnames_list)
allnames.columns = ["name", 'gender', 'count']
allnames.shape

(2020863, 3)

In [316]:
# allnames.dtypes
## drop gender and aggregate the counts groupby names
# allnames.drop(['gender'], axis = 1, inplace = True)\
#     .groupby('name').agg({'count': 'sum'})
# allnames.describe()

,name,count
1076,Jacqueli,157
1099,Cassandr,152
14787,Christop,1082


In [319]:
# drop duplicates and keep only the names with count larger than 100
allnames.drop_duplicates(subset='name',
                         keep = False, inplace= True)
allnames = allnames[allnames['count']>=10]
allnames.shape

(375, 3)

replace every name in the plot with character


In [233]:
# # ALERT: THIS CELL TAKES VERY LONG TO RUN
# # The name is a list from the names data of yob1980.txt
# for i in movies_df_sub.index:
#     for tag in name:
#         movies_df_sub.Plot[i] = re.sub(r'\b' + tag + r'\b', 'Character', movies_df_sub.Plot[i])
#
# # Plot.to_csv('test.csv', index=False)

TypeError: 'Series' object is not callable

In [320]:
# ALERT: THIS CELL TAKES EVEN LONGER TO RUN
# The name is a list from the all names data
name = allnames.name.to_list()
for i in movies_df_sub.index:
    for tag in name:
        movies_df_sub.Plot[i] = re.sub(r'\b' + tag + r'\b', 'Character', movies_df_sub.Plot[i])

# Plot.to_csv('test.csv', index=False)

TF-IDF with the updated plot summaries


In [ ]:
tfidf_matrix = pipe.fit_transform([x for x in movies_df_sub['Plot']])
print(tfidf_matrix.shape)

from sklearn.cluster import KMeans

num_clusters = 23
# print(num_clusters)
km = KMeans(n_clusters=num_clusters)

km.fit(tfidf_matrix)

clusters_tfidf = km.labels_.tolist()

import joblib

joblib.dump(km, 'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

films = {'title': movies_df_sub["Title"], 'Release_year':  movies_df_sub["Release Year"],
         'Plot':  movies_df_sub['Plot'], 'cluster': clusters_tfidf,
         'Director':  movies_df_sub['Director'], 'Genre':  movies_df_sub['Genre']}
# print(films)
frame = pd.DataFrame(data=films, columns=['title', 'cluster', 'Genre', 'Release_year', 'Director'])
# print(frame)
print(frame['cluster'].value_counts())
# frame.to_csv(r'~/Dropbox/ImmInnHollywood/data/wiki_film_plot/plot_cluster.csv', index=False)

LDA with the updated plot summaries

In [ ]:
processed_docs = movies_df_sub['Plot'].map(preprocess)
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 15:
        break

dictionary.filter_extremes(no_below=20, no_above=0.1, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

for doc in corpus_tfidf:
    pprint(doc)
    break

# lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=23, id2word=dictionary, passes=2, workers=4)
# for idx, topic in lda_model.print_topics(-1):
#     print('Topic: {} \nWords: {}'.format(idx, topic))

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=23, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

mdiff, annotation = lda_model_tfidf.diff(lda_model_tfidf)
for index, score in sorted(lda_model_tfidf[bow_corpus[4]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))

### Visualize the plot text (word cloud)

- Overall topic
- US-born director
- Foreign-born director
- Genre

## Import the data with other movie attributes

In [169]:
movies_other = pd.read_csv('~/Dropbox/Course/PSTAT234/spr21/pstat234-final-project/df_pstat234.csv')
movies_other.head(5)


,Name,Dirct1,Prodc1,Distributor,Cinematographer1,Actor1,Actor2,Actor3,Actor4,Actor5,Genre,us.year,us.quarter,domestic_bo,internaional_bo,budget_nielsen,budget_numbers,director_imm,director_continent,director_country,director_gender,director_race,international_bo
0,300 Year Weekend,Victor Stoloff,NaN,Cinerama Releasing Corporation,Joseph Brun,Michael Tolan,Sharon Laughlin,Roy Cooper,Gabriel Dell,M'el Dowd,Drama,1970,NaN,NaN,NaN,NaN,NaN,Foreign-born,NaN,Russian,NaN,NaN,NaN
1,A Bullet For Pretty Boy,Larry Buchanan,Larry Buchanan,NaN,NaN,Fabian,Jocelyn Lane,Astrid Warner,Michael Haynes,Adam Roarke,Action,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A Day at the Beach,Simon Hesera,Roman Polanski,Paramount Pictures,Gilbert Taylor,Mark Burns,Beatrice Edney,Peter Sellers,NaN,NaN,Drama,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A Man Called Sledge,Vic Morrow,Harry Bloom,Sony Pictures,Luigi Kuveiller,James Garner,Dennis Weaver,Claude Akins,John Marley,Laura Antonelli,Action,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A Run For the Money,Robert Canton,Robert Canton,NaN,Ken Van Sickle,Fred Dennis,Kim Pope,Bob Walden,Lisa Emmet,Eileen Dietz,Crime,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Import the data with big star names
*Source: https://www.imdb.com/list/ls058011111/?sort=list_order,asc&mode=detail*
*Scraped, code attached separately*

In [158]:
topstar = pd.read_csv('~/Dropbox/Course/PSTAT234/spr21/pstat234-final-project/topstar.csv')
topstar.head(5)

,Name
0,Robert De Niro
1,Jack Nicholson
2,Marlon Brando
3,Denzel Washington
4,Katharine Hepburn


## merge the data frames
- other movie attributes (*movies_other*)
- plot clustering with updated plots (*frame*)
- top star indicator (*topstar*)

,Name,Dirct1,Prodc1,Distributor,Cinematographer1,Actor1,Actor2,Actor3,Actor4,Actor5,Genre,us.year,us.quarter,domestic_bo,internaional_bo,budget_nielsen,budget_numbers,director_imm,director_continent,director_country,director_gender,director_race,international_bo
0,300 Year Weekend,Victor Stoloff,NaN,Cinerama Releasing Corporation,Joseph Brun,Michael Tolan,Sharon Laughlin,Roy Cooper,Gabriel Dell,M'el Dowd,Drama,1970,NaN,NaN,NaN,NaN,NaN,Foreign-born,NaN,Russian,NaN,NaN,NaN
1,A Bullet For Pretty Boy,Larry Buchanan,Larry Buchanan,NaN,NaN,Fabian,Jocelyn Lane,Astrid Warner,Michael Haynes,Adam Roarke,Action,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A Day at the Beach,Simon Hesera,Roman Polanski,Paramount Pictures,Gilbert Taylor,Mark Burns,Beatrice Edney,Peter Sellers,NaN,NaN,Drama,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A Man Called Sledge,Vic Morrow,Harry Bloom,Sony Pictures,Luigi Kuveiller,James Garner,Dennis Weaver,Claude Akins,John Marley,Laura Antonelli,Action,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A Run For the Money,Robert Canton,Robert Canton,NaN,Ken Van Sickle,Fred Dennis,Kim Pope,Bob Walden,Lisa Emmet,Eileen Dietz,Crime,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A.K.A. Cassius Clay,Jim Jacobs,William Cayton,NaN,Isidore Mankofsky,NaN,NaN,NaN,NaN,NaN,Documentary,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,About Me: A Musical,Robert Frank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Experimental,1971,NaN,NaN,NaN,NaN,NaN,Native-born,North America (US),US,NaN,NaN,NaN
7,Adam at 6 A.M.,Robert Scheerer,Rick Rosenberg,National General Pictures,Charles Rosher Jr.,Michael Douglas,Lee Purcell,Joe Don Baker,Grayson Hall,Charles Aidman,Drama,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Airport,George Seaton,Ross Hunter,Universal,Ernest Laszlo,Burt Lancaster,Dean Martin,Jean Seberg,Jacqueline Bisset),George Kennedy,Thriller,1970,NaN,NaN,NaN,10000000.0,NaN,Native-born,North America (US),US,NaN,NaN,NaN
9,Alex in Wonderland,Paul Mazursky,Larry Tucker,NaN,Laszlo Kovacs,Donald Sutherland,Ellen Burstyn,Meg Mazursky,Glenna Sergent,Viola Spolin,Comedy,1970,NaN,NaN,NaN,NaN,NaN,Native-born,North America (US),US,NaN,NaN,NaN


generate top star indicator for each movie if one of the main actors is on the list of top star from the above source.

In [176]:
%%R

movies_other = read.csv('~/Dropbox/Course/PSTAT234/spr21/pstat234-final-project/df_pstat234.csv')
movies_other[1, ]

              Name         Dirct1 Prodc1                    Distributor
1 300 Year Weekend Victor Stoloff   <NA> Cinerama Releasing Corporation
  Cinematographer1        Actor1          Actor2     Actor3       Actor4
1      Joseph Brun Michael Tolan Sharon Laughlin Roy Cooper Gabriel Dell
     Actor5 Genre us.year us.quarter domestic_bo internaional_bo budget_nielsen
1 M'el Dowd Drama    1970         NA          NA              NA             NA
  budget_numbers director_imm director_continent director_country
1             NA Foreign-born               <NA>         Russian 
  director_gender director_race international_bo
1            <NA>          <NA>               NA


## Exploratory analysis

TBU

*with figures and summary statistics*

In [43]:
%%R
# -i df -w 5 -h 5 --units in -r 200
# import df from global environment
# make default figure size 5 by 5 inches with 200 dpi resolution

NULL


# Regression analysis


In [188]:
%%R
fit1 = feols(log(domestic_bo + sqrt(1+domestic_bo^2)) ~ director_imm + director_gender + log(1+budget_nielsen)  | us.year + Genre + Distributor, data =movies_other)
summary(fit1)

# fit2 = feols(log(international_bo + sqrt(1+international_bo^2)) ~ director_imm + log(1+budget_numbers) | us.year + Genre + Distributor, movies_other)
# summary(fit2)

R[write to console]: NOTE: 21,559 observations removed because of NA values (LHS: 15,404, RHS: 21,248, Fixed-effects: 7,917).

R[write to console]: Warning:
R[write to console]:  In print.fixest(object, useS4 = FALSE):
 'useS4' is not a valid argument of function print.fixest (fyi, some of
its main arguments are 'n', 'type', 'se' and 'cluster').



OLS estimation, Dep. Var.: log(domestic_bo + sqrt(1 + domestic_bo^2))
Observations: 3,319 
Fixed-effects: us.year: 40,  Genre: 21,  Distributor: 268
Standard-errors: Clustered (us.year) 
                        Estimate Std. Error   t value    Pr(>|t|)    
director_immNative-born 0.329304   0.067021  4.913500 1.60000e-05 ***
director_genderMale     0.000707   0.142389  0.004966 9.96063e-01    
log(1 + budget_nielsen) 0.700227   0.052000 13.466000 3.02000e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Log-likelihood: -5,967.91   Adj. R2: 0.64172 
                          R2-Within: 0.16848 


In [189]:
%%R
fit11st = feols(log(domestic_bo + sqrt(1+domestic_bo^2)) ~ log(1+budget_nielsen) + director_gender | us.year + Genre + Distributor, data =movies_other)
summary(fit11st)

R[write to console]: NOTE: 21,330 observations removed because of NA values (LHS: 15,404, RHS: 20,927, Fixed-effects: 7,917).

R[write to console]: Warning:
R[write to console]:  In print.fixest(object, useS4 = FALSE):
 'useS4' is not a valid argument of function print.fixest (fyi, some of
its main arguments are 'n', 'type', 'se' and 'cluster').



OLS estimation, Dep. Var.: log(domestic_bo + sqrt(1 + domestic_bo^2))
Observations: 3,548 
Fixed-effects: us.year: 40,  Genre: 21,  Distributor: 315
Standard-errors: Clustered (us.year) 
                        Estimate Std. Error   t value    Pr(>|t|)    
log(1 + budget_nielsen) 0.676808   0.050244 13.471000 2.98000e-16 ***
director_genderMale     0.016215   0.134817  0.120277 9.04881e-01    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Log-likelihood: -6,415.09   Adj. R2: 0.66227 
                          R2-Within: 0.16042 


In [190]:
%%R
fit11 = lm(resid(fit11st) ~ director_imm, movies_other)
summary(fit11)
# fit11cl = coeftest(fit11, vcov. = vcovCL, cluster = ~cluster)
# fit11cl

# fit21st = feols(log(intNationbo_numbers + sqrt(1+intNationbo_numbers^2)) ~ log(1+budget_numbers)| us.year + Genre.Recode + Distributor, df)
# summary(fit21st)
# fit22 = lm(resid(fit21st) ~ dir1.imm, df)
# fit22cl = coeftest(fit22, vcov. = vcovCL, cluster = ~cluster)
# fit22cl



Call:
lm(formula = resid(fit11st) ~ director_imm, data = movies_other)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.2280 -0.5893  0.1672  0.8513  6.3669 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)             -0.16719    0.04340  -3.853 0.000119 ***
director_immNative-born  0.27472    0.05363   5.122 3.19e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.469 on 3317 degrees of freedom
  (21559 observations deleted due to missingness)
Multiple R-squared:  0.007848,	Adjusted R-squared:  0.007548 
F-statistic: 26.24 on 1 and 3317 DF,  p-value: 3.193e-07



# References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2

# Appendix

example showing how the preprocessing works

In [30]:
doc_sample = movies_df_sub[movies_df_sub['Title'] == 'Madagascar'].values[0][7]
print(doc_sample)
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))


At the Central Park Zoo, Marty the zebra is celebrating his tenth birthday, but has grown bored with his daily routine and longs to experience the wild. Marty's best friend is Alex the lion, who enjoys showing off for the public and his celebrity status as "the King of New York City". Alex attempts to cheer Marty up, but Marty, still unsatisfied, gets some tips from the zoo's penguins—Skipper, Kowalski, Rico, and Private—who are trying to escape the zoo, and follows them out. Alex, Melman the giraffe, and Gloria the hippopotamus pursue Marty in an attempt to convince him to return. The four, along with the penguins and two chimpanzees named Mason and Phil, find themselves at Grand Central Station, where they are quickly sedated via tranquillizer gun when Alex's attempt to communicate with humans is mistaken for aggression. The zoo, under pressure from anti-captivity activists, is forced to ship the escaped animals by sea to a Kenyan wildlife preserve. During their travels, the penguins

testing with name replacement

In [ ]:
miniplot = movies_df_sub.Plot[5]
# miniplot.head()

In [ ]:
# print(miniplot)
# tags = ['Alice', 'John']
for tag in names:
    miniplot = miniplot.replace(tag, 'Character')

print(miniplot)

In [ ]:
for tag in name:
    miniplot = re.sub(r'\b' + tag + r'\b', 'Character', miniplot)
print(miniplot)

In [ ]:
Plot = movies_df_sub.Plot.sample(10)
for i in Plot.index:
    for tag in name:
        Plot[i] = re.sub(r'\b' + tag + r'\b', 'Character', Plot[i])
print(Plot)
# Plot.to_csv('test.csv', index=False)

In [ ]:
'Grace' in name